In [ ]:
import pathlib
import json
import lzma

import pandas as pd

In [ ]:
# Makes it so any changes in pymedphys is automatically
# propagated into the notebook without needing a kernel reset.
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

In [ ]:
import pymedphys
import pymedphys._utilities.filehash

from prototyping import *

In [ ]:
icom_directory = pathlib.Path(r'\\physics-server\iComLogFiles\patients')
monaco_directory = pathlib.Path(r'\\monacoda\FocalData\RCCC\1~Clinical')
output_directory = pathlib.Path(r'S:\Physics\Patient Specific Logfile Fluence')

pdf_directory = pathlib.Path(r'P:\Scanned Documents\RT\PhysChecks\Logfile PDFs')

## Patient ID Cell

In [ ]:
patient_id = '008566'

percent_deviation = 2
mm_dist_threshold = 0.5

## Available Monaco Plans and Mosaiq Sites

### iCOM Deliveries

In [ ]:
icom_deliveries = list(icom_directory.glob(f'{patient_id}_*/*.xz'))
icom_deliveries

### Monaco Plans

In [ ]:
all_tel_paths = list(monaco_directory.glob(f'*~{patient_id}/plan/*/tel.1'))
all_tel_paths

## Custom Monaco/iCOM Alignment

In [ ]:
monaco_plan_name = 'WHOLEBRAIN'
icom_delivery = '20200213_091126'

In [ ]:
tel_path = list(monaco_directory.glob(f'*~{patient_id}/plan/{monaco_plan_name}/tel.1'))[-1]
tel_path

In [ ]:
icom_path = list(icom_directory.glob(f'{patient_id}_*/{icom_delivery}.xz'))[-1]
icom_path

In [ ]:
with lzma.open(icom_path, 'r') as f:
    icom_stream = f.read()

In [ ]:
delivery_icom = pymedphys.Delivery.from_icom(icom_stream)
mudensity_icom = delivery_icom.mudensity()

In [ ]:
mudensity_tel = get_mu_density_from_file(tel_path)

In [ ]:
gamma = calc_gamma(to_tuple(mudensity_tel), to_tuple(mudensity_icom))

results_dir = output_directory.joinpath(patient_id, tel_path.parent.name, icom_path.stem)
results_dir.mkdir(exist_ok=True, parents=True)

header_text = (
    f"Patient ID: {patient_id}\n"
    f"Plan Name: {tel_path.parent.name}\n"
)

footer_text = (
    f"tel.1 file path: {str(tel_path)}\n"
    f"icom file path: {str(icom_path)}\n"
    f"results path: {str(results_dir)}"
)

png_filepath = str(results_dir.joinpath("result.png").resolve())
pdf_filepath = str(pdf_directory.joinpath(f"{patient_id}.pdf").resolve())

fig = plot_and_save_results(
    mudensity_tel, mudensity_icom, 
    gamma, png_filepath, pdf_filepath, 
    header_text=header_text, footer_text=footer_text
)

fig.tight_layout()
plt.savefig(png_filepath, dpi=300)
plt.show()

!magick convert "{png_filepath}" "{pdf_filepath}"